# <h2> Project :  Part 1 </h2>   
<h2>Ala Bouslama  </h2>                                            


In [165]:
#Import packages
import pyspark
from pyspark import SparkContext
from matplotlib import pyplot as plt
import seaborn as sb

In [166]:
#create a SparkSession from pyspark.sql
spark = pyspark.sql.SparkSession.builder.appName("Spark-sales-dfSQL").getOrCreate()

In [167]:
#Read data
df  =  spark.read.csv('project_data_real_estate_toulouse_and_suburbs.csv', header=True, sep=';') 
df.show(3)


+----------+--------+------+--------+-------------------+-------+------+-------------+----------+--------+
|  SaleDate|SaleType| Price|PostCode|               City|NbRooms|NbLots|ResidenceType|LivingArea|LandArea|
+----------+--------+------+--------+-------------------+-------+------+-------------+----------+--------+
|15/01/2019|    SALE| 95000|   31270|VILLENEUVE TOLOSANE|      2|     0|        HOUSE|        45|     505|
|21/01/2019|    SALE|261700|   31270|VILLENEUVE TOLOSANE|      4|     0|        HOUSE|        87|     311|
|03/01/2019|    SALE|101000|   31270|VILLENEUVE TOLOSANE|      2|     1|    APARTMENT|        44|       0|
+----------+--------+------+--------+-------------------+-------+------+-------------+----------+--------+
only showing top 3 rows



Change data Types and Update the structure 


In [169]:


from pyspark.sql.types import *

sales_schema =StructType ( [StructField("SaleDate", DateType(), True),
                           StructField("SaleType", StringType(), True),
                           StructField("Price", DoubleType(), True),
                           StructField("PostCode", StringType(), True),
                           StructField("City", StringType(), True),
                           StructField("NbRooms", IntegerType(), True),
                           StructField("NbLots", IntegerType(), True),
                           StructField("ResidenceType", StringType(), True),
                           StructField("LivingArea", IntegerType(), True),
                           StructField("LandArea", IntegerType(), True)]
                            )

In [170]:
#Read new dataframe with updated structure
df = spark.read.csv('project_data_real_estate_toulouse_and_suburbs.csv', header=True, sep=';', dateFormat='dd/MM/YYYY', schema=sales_schema)
df.show()

+----------+--------------------+--------+--------+-------------------+-------+------+-------------+----------+--------+
|  SaleDate|            SaleType|   Price|PostCode|               City|NbRooms|NbLots|ResidenceType|LivingArea|LandArea|
+----------+--------------------+--------+--------+-------------------+-------+------+-------------+----------+--------+
|2019-01-15|                SALE| 95000.0|   31270|VILLENEUVE TOLOSANE|      2|     0|        HOUSE|        45|     505|
|2019-01-21|                SALE|261700.0|   31270|VILLENEUVE TOLOSANE|      4|     0|        HOUSE|        87|     311|
|2019-01-03|                SALE|101000.0|   31270|VILLENEUVE TOLOSANE|      2|     1|    APARTMENT|        44|       0|
|2019-01-11|                SALE| 59500.0|   31270|VILLENEUVE TOLOSANE|      1|     1|    APARTMENT|        26|       0|
|2019-01-15|                SALE|179000.0|   31270|VILLENEUVE TOLOSANE|      4|     0|        HOUSE|        95|     263|
|2019-01-15|                SALE

In [9]:
df.count()

80118

In [10]:
type(df)

pyspark.sql.dataframe.DataFrame

In [172]:
#Variable's outliers  : 

df.summary("count", "min", "50%" ,"25%", "75%", "max").show()


+-------+--------------------+-----------+--------+-------------------+-------+------+-------------+----------+--------+
|summary|            SaleType|      Price|PostCode|               City|NbRooms|NbLots|ResidenceType|LivingArea|LandArea|
+-------+--------------------+-----------+--------+-------------------+-------+------+-------------+----------+--------+
|  count|               77105|      77105|   77105|              77105|  77105| 77105|        77105|     77105|   77105|
|    min|                SALE|        0.0|   31000|         AUCAMVILLE|      1|     0|    APARTMENT|         1|       0|
|    50%|                null|   180000.0| 31300.0|               null|      3|     1|         null|        62|       0|
|    25%|                null|   122470.0| 31170.0|               null|      2|     0|         null|        43|       0|
|    75%|                null|   280000.0| 31500.0|               null|      4|     2|         null|        82|     138|
|    max|SALE BEFORE COMPL...|3.

As we can see the outliers of our variables columns ( For example , the average price is 180000 and the maximum is 33427218 , The minimum number of rooms is 1 and the average is 41 ...)

# Average Price per location  

In this part , we can also identify high cost and low cost cities 

<B> High cost cities


In [176]:
average_price_per_city = df.groupBy('City').agg({'Price':'avg'}).orderBy('avg(Price)', ascending=False)
average_price_per_city.show()


+--------------------+------------------+
|                City|        avg(Price)|
+--------------------+------------------+
|          SAINT-JEAN|  863260.856744186|
|              PIBRAC| 836622.0687732341|
|            TOULOUSE| 634151.5455853839|
|             CUGNAUX| 579626.7678571428|
|    VIEILLE TOULOUSE|          570051.5|
|             BLAGNAC|507254.94533762056|
|          AUCAMVILLE| 472807.3695652174|
|RAMONVILLE-SAINT-...|434089.01343873516|
| VILLENEUVE TOLOSANE| 353282.4527363184|
|       TOURNEFEUILLE|347073.55194527574|
|  PLAISANCE DU TOUCH|335517.85353535356|
|           COLOMIERS|323835.84222222224|
|               BALMA|306818.94100074685|
|              LABEGE| 303852.6934306569|
|             L UNION|297758.53948832036|
|        CORNEBARRIEU|296385.54285714286|
|  AUZEVILLE-TOLOSANE|289053.77040816325|
|SAINT-ORENS-DE-GA...|285969.78851412947|
|               MURET|279293.25756457564|
|    CASTANET TOLOSAN| 266280.7614607615|
+--------------------+------------

The highest cost cities are SAINT-JEAN , Pibrac , and Toulouse

<b> Low cost cities 

In [177]:
#Low cost cities and their postCodes
average_price_per_city2 = df.groupBy('City','PostCode').agg({'Price':'avg'}).orderBy('avg(Price)', ascending=True)
average_price_per_city2.show()


+--------------------+--------+------------------+
|                City|PostCode|        avg(Price)|
+--------------------+--------+------------------+
|                null|    null|              null|
|          FENOUILLET|   31150|195298.22702702702|
|            LEGUEVIN|   31490|201879.80507131538|
|  PORTET SUR GARONNE|   31120|227264.96697247706|
|           BEAUZELLE|   31700|228428.07719298246|
|          ESCALQUENS|   31750|255022.96996996997|
|   QUINT-FONSEGRIVES|   31130| 262555.5038560411|
|    CASTANET TOLOSAN|   31320| 266280.7614607615|
|               MURET|   31600|279293.25756457564|
|SAINT-ORENS-DE-GA...|   31650|285969.78851412947|
|  AUZEVILLE-TOLOSANE|   31320|289053.77040816325|
|        CORNEBARRIEU|   31700|296385.54285714286|
|             L UNION|   31240|297758.53948832036|
|              LABEGE|   31670| 303852.6934306569|
|               BALMA|   31130|306818.94100074685|
|           COLOMIERS|   31770|323835.84222222224|
|  PLAISANCE DU TOUCH|   31830|

The lowest cost cities are FENOUILLET and LEGUEVIN  ( prices are 4 Times less than SAINT-Jean ) 

In [179]:
# Add year column 
df1 = df.withColumn("year", pyspark.sql.functions.split(df['SaleDate'], '-')[0])
df1.show(4)

+----------+--------+--------+--------+-------------------+-------+------+-------------+----------+--------+----+
|  SaleDate|SaleType|   Price|PostCode|               City|NbRooms|NbLots|ResidenceType|LivingArea|LandArea|year|
+----------+--------+--------+--------+-------------------+-------+------+-------------+----------+--------+----+
|2019-01-15|    SALE| 95000.0|   31270|VILLENEUVE TOLOSANE|      2|     0|        HOUSE|        45|     505|2019|
|2019-01-21|    SALE|261700.0|   31270|VILLENEUVE TOLOSANE|      4|     0|        HOUSE|        87|     311|2019|
|2019-01-03|    SALE|101000.0|   31270|VILLENEUVE TOLOSANE|      2|     1|    APARTMENT|        44|       0|2019|
|2019-01-11|    SALE| 59500.0|   31270|VILLENEUVE TOLOSANE|      1|     1|    APARTMENT|        26|       0|2019|
+----------+--------+--------+--------+-------------------+-------+------+-------------+----------+--------+----+
only showing top 4 rows



# Difference between types of sale 

In [180]:
#Spliting data into two ( ' Sales' and 'Sales before completion') to see the difference 
# Sale Type Data 

sale = df1.filter(df.SaleType=='SALE')
sale.show(3)


+----------+--------+--------+--------+-------------------+-------+------+-------------+----------+--------+----+
|  SaleDate|SaleType|   Price|PostCode|               City|NbRooms|NbLots|ResidenceType|LivingArea|LandArea|year|
+----------+--------+--------+--------+-------------------+-------+------+-------------+----------+--------+----+
|2019-01-15|    SALE| 95000.0|   31270|VILLENEUVE TOLOSANE|      2|     0|        HOUSE|        45|     505|2019|
|2019-01-21|    SALE|261700.0|   31270|VILLENEUVE TOLOSANE|      4|     0|        HOUSE|        87|     311|2019|
|2019-01-03|    SALE|101000.0|   31270|VILLENEUVE TOLOSANE|      2|     1|    APARTMENT|        44|       0|2019|
+----------+--------+--------+--------+-------------------+-------+------+-------------+----------+--------+----+
only showing top 3 rows



In [181]:
#SALE BEFORE COMPLETION
sale_before_completion = df1.filter(df.SaleType=='SALE BEFORE COMPLETION')
sale_before_completion.show(3)


+----------+--------------------+--------+--------+-------------------+-------+------+-------------+----------+--------+----+
|  SaleDate|            SaleType|   Price|PostCode|               City|NbRooms|NbLots|ResidenceType|LivingArea|LandArea|year|
+----------+--------------------+--------+--------+-------------------+-------+------+-------------+----------+--------+----+
|2019-01-16|SALE BEFORE COMPL...|214300.0|   31270|VILLENEUVE TOLOSANE|      3|     0|        HOUSE|        66|     164|2019|
|2018-02-15|SALE BEFORE COMPL...|155000.0|   31270|VILLENEUVE TOLOSANE|      2|     1|    APARTMENT|        44|       0|2018|
|2018-02-28|SALE BEFORE COMPL...|213009.0|   31270|VILLENEUVE TOLOSANE|      3|     1|    APARTMENT|        63|       0|2018|
+----------+--------------------+--------+--------+-------------------+-------+------+-------------+----------+--------+----+
only showing top 3 rows



In [47]:
#Here , we want to see the average LIVING AREA per residence type of the dataset ('residencetype=SALE')
average_livingarea_of_appartments_per_city = sale.groupBy('ResidenceType').agg({'Price':'avg','LivingArea': 'avg'}).orderBy('avg(Price)', ascending=False)
average_livingarea_of_appartments_per_city.show()

+-------------+------------------+-----------------+
|ResidenceType|   avg(LivingArea)|       avg(Price)|
+-------------+------------------+-----------------+
|    APARTMENT| 56.09170754202825|677190.8301052279|
|        HOUSE|108.26817690181468| 452817.812822978|
+-------------+------------------+-----------------+



In [48]:
#Here , we want to see the LIVING AREA per residence type of the dataset ('residencetype=SALE BOFORE COMPLETION')
average_livingarea_of_appartments_per_city = sale_before_completion.groupBy('ResidenceType').agg({'Price':'avg','LivingArea': 'avg'}).orderBy('avg(Price)', ascending=False)
average_livingarea_of_appartments_per_city.show()

+-------------+-----------------+------------------+
|ResidenceType|  avg(LivingArea)|        avg(Price)|
+-------------+-----------------+------------------+
|        HOUSE|82.13739376770538| 305075.5807365439|
|    APARTMENT|53.33557381130197|208787.07559538627|
+-------------+-----------------+------------------+



The average price of appartments sold completeted is around 3 times the price of those not completed , 
Concerning the houses , the average price of houses sold complete is 1.5 times those not completed.  

# Evolution of prices per type of sale

In [56]:
Evo_of_price_of_saleType = sale.groupBy('year').agg({'price':'avg'}).orderBy('year', ascending=False)
Evo_of_price_of_saleType.show()

+----+------------------+
|year|        avg(price)|
+----+------------------+
|2019| 570175.4586724543|
|2018|296564.36668928084|
|2017| 585274.4765889375|
|2016| 528384.1574317134|
|2015|1118488.1027601778|
+----+------------------+



<b>There is a big increase of sale type from 2017 to 2019 (it has been multiplied 4 times from 2017 to 2018 and 2 times from 2018 to 2019 ) 

In [58]:
Evo_of_price_of_sale_before_completion_Type = sale_before_completion.groupBy('year').agg({'price':'avg'}).orderBy('year', ascending=False)
Evo_of_price_of_sale_before_completion_Type.show()

+----+------------------+
|year|        avg(price)|
+----+------------------+
|2019| 222149.6220472441|
|2018|218519.98218673217|
|2017| 223289.5234561357|
|2016| 214873.3239149432|
|2015|202155.26396206534|
+----+------------------+



No important evolution in the last years for the price of this type of sale

#  Differences between house and appartment

Split data into 2 ( appartments and houses )


In [72]:
#Appartments
appartments = df1.filter(df.ResidenceType=='APARTMENT')
appartments.count()

62488

In [182]:
#houses 
houses = df1.filter(df.ResidenceType=='HOUSE')
houses.count()

17630

<B> Adding price Per square for appartments column and another one for houses

In [184]:

import pyspark.sql.functions as F



In [188]:
df1 = df1.withColumn("price_per_square", F.col("Price") / F.col("LivingArea"))
df1.show(5)

+----------+--------+--------+--------+-------------------+-------+------+-------------+----------+--------+----+------------------+
|  SaleDate|SaleType|   Price|PostCode|               City|NbRooms|NbLots|ResidenceType|LivingArea|LandArea|year|  price_per_square|
+----------+--------+--------+--------+-------------------+-------+------+-------------+----------+--------+----+------------------+
|2019-01-15|    SALE| 95000.0|   31270|VILLENEUVE TOLOSANE|      2|     0|        HOUSE|        45|     505|2019|2111.1111111111113|
|2019-01-21|    SALE|261700.0|   31270|VILLENEUVE TOLOSANE|      4|     0|        HOUSE|        87|     311|2019|3008.0459770114944|
|2019-01-03|    SALE|101000.0|   31270|VILLENEUVE TOLOSANE|      2|     1|    APARTMENT|        44|       0|2019|2295.4545454545455|
|2019-01-11|    SALE| 59500.0|   31270|VILLENEUVE TOLOSANE|      1|     1|    APARTMENT|        26|       0|2019|2288.4615384615386|
|2019-01-15|    SALE|179000.0|   31270|VILLENEUVE TOLOSANE|      4|  

In [201]:
appartments_with_pricepersquare = appartments.withColumn("price_per_square", F.col("Price") / F.col("LivingArea"))
appartments_with_pricepersquare.show(3)

+----------+--------+--------+--------+-------------------+-------+------+-------------+----------+--------+----+------------------+
|  SaleDate|SaleType|   Price|PostCode|               City|NbRooms|NbLots|ResidenceType|LivingArea|LandArea|year|  price_per_square|
+----------+--------+--------+--------+-------------------+-------+------+-------------+----------+--------+----+------------------+
|2019-01-03|    SALE|101000.0|   31270|VILLENEUVE TOLOSANE|      2|     1|    APARTMENT|        44|       0|2019|2295.4545454545455|
|2019-01-11|    SALE| 59500.0|   31270|VILLENEUVE TOLOSANE|      1|     1|    APARTMENT|        26|       0|2019|2288.4615384615386|
|2018-01-08|    SALE|123000.0|   31270|VILLENEUVE TOLOSANE|      3|     1|    APARTMENT|        56|       0|2018|2196.4285714285716|
+----------+--------+--------+--------+-------------------+-------+------+-------------+----------+--------+----+------------------+
only showing top 3 rows



<b> Difference of price per PostCode

In [190]:
avg_price_per_square_forAppartments = appartments_with_pricepersquare.groupby('PostCode').agg({'price_per_square':'avg'}).orderBy('avg(price_per_square)', ascending=False)
avg_price_per_square_forAppartments.show()


+--------+---------------------+
|PostCode|avg(price_per_square)|
+--------+---------------------+
|   31400|   22323.380059533574|
|   31100|   20847.980459397866|
|   31300|   13925.717410471525|
|   31270|   11541.471487764398|
|   31500|    11370.64177038219|
|   31140|   10651.709807646346|
|   31700|    9051.176956767044|
|   31000|    8721.906493357686|
|   31200|    8375.292606104667|
|   31520|   8078.5330840044635|
|   31770|    7877.562609509424|
|   31600|    7717.693642026942|
|   31170|    6030.439058117998|
|   31670|    4279.987884045257|
|   31240|     4000.07608288499|
|   31320|   3952.0634293867415|
|   31650|   3364.0620736246065|
|   31130|    3307.661097809217|
|   31820|    3226.379831424579|
|   31120|   2958.8143588618336|
+--------+---------------------+
only showing top 20 rows



<b>31400 ,31100  are the most expensive PostCode Places for buying an appartment with an average price per square higher than 20000 

In [192]:
houses_with_pricepersquare = houses.withColumn("price_per_square", F.col("Price") / F.col("LandArea"))
houses_with_pricepersquare.show(2)


+----------+--------+--------+--------+-------------------+-------+------+-------------+----------+--------+----+------------------+
|  SaleDate|SaleType|   Price|PostCode|               City|NbRooms|NbLots|ResidenceType|LivingArea|LandArea|year|  price_per_square|
+----------+--------+--------+--------+-------------------+-------+------+-------------+----------+--------+----+------------------+
|2019-01-15|    SALE| 95000.0|   31270|VILLENEUVE TOLOSANE|      2|     0|        HOUSE|        45|     505|2019|188.11881188118812|
|2019-01-21|    SALE|261700.0|   31270|VILLENEUVE TOLOSANE|      4|     0|        HOUSE|        87|     311|2019| 841.4790996784566|
+----------+--------+--------+--------+-------------------+-------+------+-------------+----------+--------+----+------------------+
only showing top 2 rows



In [193]:
avg_price_per_square_forHouses =houses_with_pricepersquare.groupby('PostCode').agg({'price_per_square':'avg'}).orderBy('avg(price_per_square)', ascending=False)
avg_price_per_square_forHouses.show()

+--------+---------------------+
|PostCode|avg(price_per_square)|
+--------+---------------------+
|   31000|   3929.9443308600776|
|   31400|   2101.8038395462468|
|   31500|    1616.587496577286|
|   31300|   1458.3146672694459|
|   31130|   1287.2186637011011|
|   31200|     1269.83500287272|
|   31100|   1049.2727045764814|
|   31650|    955.6543422143728|
|   31240|    927.7659066344393|
|   31700|    868.6954524652988|
|   31320|    851.0163701538922|
|   31770|    782.5110048190816|
|   31140|    776.6712597590027|
|   31830|    755.0444427982449|
|   31820|    740.2166061326623|
|   31270|    734.5379680405911|
|   31520|    699.6524574867327|
|   31170|    696.5839627384345|
|   31750|     659.054681706938|
|   31670|    642.9162438033767|
+--------+---------------------+
only showing top 20 rows



Concerning the price of houses , in 31000 and 31400 we have the highest prices of M² ( 3929e in 31000 and 2101 in 31400 ) 


<b> 31 000 has the highest price for buying a house , and the 8th highest for buying an appartment )
    ==> Clients prefer to buy houses in the center of Toulouse 31000 rather than buying an appartment

# Evolution of prices per Residence Type ( house , app ) per year 

In [194]:
evo=df1.groupby('ResidenceType','year').agg({'price_per_square':'avg'}).orderBy('year', ascending=False)
evo.show()

+-------------+----+---------------------+
|ResidenceType|year|avg(price_per_square)|
+-------------+----+---------------------+
|    APARTMENT|2019|    22956.02133744783|
|        HOUSE|2019|     3647.30923723466|
|        HOUSE|2018|   3871.9843523421737|
|    APARTMENT|2018|    5050.254087396504|
|        HOUSE|2017|    3462.928915582423|
|    APARTMENT|2017|    11104.29176950227|
|    APARTMENT|2016|    9926.438685041774|
|        HOUSE|2016|   3666.3631270054857|
|        HOUSE|2015|    8772.945276515396|
|    APARTMENT|2015|    16945.01316351537|
|         null|null|                 null|
+-------------+----+---------------------+



<b> The price_per_sqaure  of appartments increased a lot  from 2018 to 2019 while the price of M² is almost constant from 2018 to 2019 (increase of 200euro)

# Differences between Toulouse and  its suburbs 
 


In this step , we want to compare Toulouse with 5 suburbs which have the highest prices in our dataset

Split the data into 2 ( Toulouse and suburbs)


<b> Toulouse Data

In [203]:
toulouse = df1.filter(df.City=='TOULOUSE')
toulouse.show(5)


+----------+--------+--------+--------+--------+-------+------+-------------+----------+--------+----+------------------+
|  SaleDate|SaleType|   Price|PostCode|    City|NbRooms|NbLots|ResidenceType|LivingArea|LandArea|year|  price_per_square|
+----------+--------+--------+--------+--------+-------+------+-------------+----------+--------+----+------------------+
|2019-01-03|    SALE|380000.0|   31500|TOULOUSE|      2|     0|        HOUSE|        52|     160|2019| 7307.692307692308|
|2019-01-07|    SALE|128000.0|   31500|TOULOUSE|      5|     2|    APARTMENT|        78|       0|2019| 1641.025641025641|
|2019-01-09|    SALE|155447.0|   31500|TOULOUSE|      4|     1|    APARTMENT|        78|       0|2019|1992.9102564102564|
|2019-01-09|    SALE| 72000.0|   31500|TOULOUSE|      2|     2|    APARTMENT|        39|       0|2019|1846.1538461538462|
|2019-01-08|    SALE|108500.0|   31400|TOULOUSE|      2|     1|    APARTMENT|        45|       0|2019|2411.1111111111113|
+----------+--------+---

In [205]:
toulouse.summary("count", "min", "50%", "25%", "75%", "max").show()


+-------+--------------------+-----------+--------+--------+-------+------+-------------+----------+--------+------+------------------+
|summary|            SaleType|      Price|PostCode|    City|NbRooms|NbLots|ResidenceType|LivingArea|LandArea|  year|  price_per_square|
+-------+--------------------+-----------+--------+--------+-------+------+-------------+----------+--------+------+------------------+
|  count|               50027|      50027|   50027|   50027|  50027| 50027|        50027|     50027|   50027| 50027|             50027|
|    min|                SALE|        0.0|   31000|TOULOUSE|      1|     0|    APARTMENT|         1|       0|  2015|               0.0|
|    50%|                null|   168953.0| 31200.0|    null|      3|     1|         null|        57|       0|2017.0|3145.1612903225805|
|    25%|                null|   113000.0| 31100.0|    null|      2|     1|         null|        40|       0|2016.0|2341.8867924528304|
|    75%|                null|   270000.0| 31400

As we can see the outliers of Toulouse data  , we have 50027 sales from Toulouse and the highest price that we mentioned before is in Toulouse with an average price of 168935

In [109]:
evo=toulouse.groupby('year').agg({'price':'avg'}).orderBy('year', ascending=False)
evo.show()

+----+------------------+
|year|        avg(price)|
+----+------------------+
|2019| 694309.7682414049|
|2018|306404.16524862923|
|2017| 608159.4727398206|
|2016| 443091.7886704372|
|2015|1141088.0634241938|
+----+------------------+



<b> There is an important increase of the average price in Toulouse from 2018 to 2019 ( Doubled ) and multiplied 6 times from 2015 to 2019 

<b> Suburbs data 

In [208]:
#We choosed PIBRAC , SAINT-Jean , CUGNAUX , VIEILLE TOULOUSE , BLAGNAC as most important Suburbs because they are
toulouse_suburbs = df1.filter( (df.City == 'PIBRAC') | (df.City =='SAINT-JEAN' )  | (df.City =='CUGNAUX' )  | (df.City =='VIEILLE TOULOUSE') | (df.City =='BLAGNAC' )       )
toulouse_suburbs.show(5)

+----------+--------+--------+--------+----------------+-------+------+-------------+----------+--------+----+------------------+
|  SaleDate|SaleType|   Price|PostCode|            City|NbRooms|NbLots|ResidenceType|LivingArea|LandArea|year|  price_per_square|
+----------+--------+--------+--------+----------------+-------+------+-------------+----------+--------+----+------------------+
|2019-01-15|    SALE|550000.0|   31320|VIEILLE TOULOUSE|      8|     0|        HOUSE|       211|    2000|2019|2606.6350710900474|
|2019-01-15|    SALE|550000.0|   31320|VIEILLE TOULOUSE|      8|     0|        HOUSE|       211|    3006|2019|2606.6350710900474|
|2019-03-15|    SALE|380000.0|   31320|VIEILLE TOULOUSE|      6|     0|        HOUSE|       119|    1360|2019|  3193.27731092437|
|2019-06-06|    SALE|658050.0|   31320|VIEILLE TOULOUSE|      4|     0|        HOUSE|       105|    1057|2019| 6267.142857142857|
|2018-01-05|    SALE|734400.0|   31320|VIEILLE TOULOUSE|      6|     0|        HOUSE|     

In [209]:
evo2=toulouse_suburbs.groupby('year').agg({'price':'avg'}).orderBy('year', ascending=False)
evo2.show()


+----+------------------+
|year|        avg(price)|
+----+------------------+
|2019| 226731.1120689655|
|2018|240499.31500926497|
|2017|  535342.965453708|
|2016| 816813.2023460411|
|2015| 769436.3696771174|
+----+------------------+



<b> the avg price in toulouse's suburbs is almost constant last two years with a multiplication times 4 from 2017 to 2018

# Evolution of number of sales and comparision between Toulouse and Suburbs

In [123]:
# In all locations : 
evolution_of_Number_of_sales1=df1.groupby('ResidenceType', 'Year').count().orderBy('year', ascending=False)
evolution_of_Number_of_sales1.show()

+-------------+----+-----+
|ResidenceType|Year|count|
+-------------+----+-----+
|        HOUSE|2019| 1191|
|    APARTMENT|2019| 4518|
|    APARTMENT|2018|12336|
|        HOUSE|2018| 4301|
|        HOUSE|2017| 4463|
|    APARTMENT|2017|16389|
|    APARTMENT|2016|14588|
|        HOUSE|2016| 3856|
|        HOUSE|2015| 3819|
|    APARTMENT|2015|14657|
+-------------+----+-----+



In [124]:
# In toulouse 
evolution_of_Number_of_sales_in_toulouse=toulouse.groupby('ResidenceType', 'Year').count().orderBy('year', ascending=False)
evolution_of_Number_of_sales_in_toulouse.show()

+-------------+----+-----+
|ResidenceType|Year|count|
+-------------+----+-----+
|        HOUSE|2019|  535|
|    APARTMENT|2019| 3551|
|        HOUSE|2018| 1482|
|    APARTMENT|2018| 9272|
|    APARTMENT|2017|12028|
|        HOUSE|2017| 1574|
|        HOUSE|2016| 1314|
|    APARTMENT|2016|10371|
|        HOUSE|2015| 1286|
|    APARTMENT|2015|10384|
+-------------+----+-----+



<b> As we can see from 2018 to 2019 , The number of sales of appartments is divided to 3 , and the number of houses divided by 3 also ==> This the result of the big increase of  prices in Toulouse from 2018 to 2019 ( Doubled )

<b> from 2017 to 2018 , Prices are almost divided to 2 , and from 2018 to 2019 there is no bi evolution in prices in Suburbs of Toulouse

In [126]:
# In Suburbs 
evolution_of_Number_of_sales_in_suburbs=toulouse_suburbs.groupby('ResidenceType', 'Year').count().orderBy('year', ascending=False)
evolution_of_Number_of_sales_in_suburbs.show()

+-------------+----+-----+
|ResidenceType|Year|count|
+-------------+----+-----+
|    APARTMENT|2019|  296|
|        HOUSE|2019|  169|
|        HOUSE|2018|  684|
|    APARTMENT|2018|  972|
|        HOUSE|2017|  762|
|    APARTMENT|2017| 1503|
|        HOUSE|2016|  650|
|    APARTMENT|2016| 1569|
|        HOUSE|2015|  715|
|    APARTMENT|2015| 1572|
+-------------+----+-----+



As we can see the number of sales of houses in suburbs from 2018 to 2019 is divided into 4 ==> This is because Toulouse has the biggest market share in This region and clients are preferring to buy in Toulouse in the last 2 years